# Алгоритм решения

## Постановка задачи
Пусть дана система $$Ax = b$$ или в координатной форме 
$$\begin{cases}
		a_{11}x_1 + a_{12}x_2 + \ldots + a_{1n}x_n = b_1,\\
		a_{21}x_1 + a_{22}x_2 + \ldots + a_{2n}x_n = b_2,\\
		\vdots\\
		a_{n1}x_1 + a_{n2}x_2 + \ldots + a_{nn}x_n = b_n.
	\end{cases}.$$
    
Предполагаем, что задача является корректно поставленной. Нам необходимо:
1. найти решение системы указанным методом;
2. вычислить вектор невязки;
3. найти матрицу, обратную к матрице системы;
4. найти число обусловленности матрицы системы.

Решение системы мы будем искать как предел бесконечного вычислительного процесса. Когда строится последовательность $x^0, x^1,\ldots, x^k,\ldots$ приближения к решению $x^*$, мы должны обеспечить выполнение условия $$\lim\limits_{k\to\infty} | | x^k - x^* | | = 0.$$
В наших обозначениях $0,1,\ldots,k,\ldots$ - номер итерации. И каждый элемент является $k$-ой итерацией, или $k$-ым приближением к искомому значению $x^*$.

## Метод Зейделя
Метод простой итерации в координатной форме можно записать как $$x_i^{k+1} = \sum_{j=1}^{n} b_{ij}x^k_j + g_i,\ i=1,\ldots,n,\ k = 0,1,\ldots;\quad x_0 = g$$
	где $i$ - координата искомого вектора. Опишем метод использования вычисленных координат вектора $x^{k+1}$ для вычисления других координат этого вектора.

Фиксируем порядок вычисления компонент вектора $x^{k+1}$ в порядке возрастания номеров от $1$ до $n$ и при вычислении каждой последующей координаты будем использовать уточненные ранее предыдущей координаты. Запишем это в виде формулы 
	$$\begin{cases}
		x_1^{k+1} = b_{11}x_1^k + b_{12}x_2^k + \ldots + b_{1n}x_n^k + g_1,\\
		x_2^{k+1} = b_{21}x
        _1^{k+1} + b_{12}x_2^k + \ldots + b_{2n}x_n^k + g_2,\\
		\vdots\\
		x_n^{k+1} = b_{n1}x_1^{k+1} + b_{n2}x_2^{k+1} + \ldots + b_{nn}x_n^k + g_n
	\end{cases}\quad (1)$$
Запишем это в более компактном виде $$x_i^{k+1} = \sum_{j=1}^{i-1}b_{ij}x_j^{k+1} + \sum_{j=i}^{n} b_{ij} x_j^k + g_i,\ i = \overline{1,n}, k=0,1\ldots;\quad x_0\quad(2)$$

Итерационный метод (1) или (2) называется методом Зейделя. Для исследования его сходимости запишем формулу (2) в матричном виде. Введем в рассмотрение матрицы 
	$$H = \begin{pmatrix}
		0 & 0 & \ldots & 0 & 0\\
		b_{21} & 0 & \ldots & 0 & 0\\
		\vdots & \vdots & \ddots & \vdots & \vdots \\
		b_{n1} & b_{n2} & \dots & b_{nn-1} & 0
	\end{pmatrix},\quad F = \begin{pmatrix}
	b_{11} & b_{12}& \ldots & b_{1n-1} & b_{1n}\\
	0 & b_{22} & \dots &b_{2n-1}& b_{2n}\\
	\vdots & \vdots & \ddots & \vdots & \vdots\\
	0 & 0& \dots & 0 & b_{nn}
	\end{pmatrix}$$
    
Очевидно $B = H+F$. Тогда вместо формулы (2) мы можем записать метод Зейделя в матричном виде $$x^{k+1} = Hx^{k+1} + Fx^k + g,\ k=0,1\ldots \quad(3)$$

Для отыскания обратной матрицы вместо исходного уравнения будем решать уравнение $$AX = E,$$ где значение $X$, которое мы вычислим с помощью метода Зейделя, и будет являться матрицей обратной к матрице $A$.

## Вычисление числа обусловленности

Для вычисления числа обусловленности воспользуемся формулой, определяющей его, $$\nu(A) = | | A| | \cdot | | A^{-1} | |.$$
При решении будет вычислять его, используя сферическую норму матрицы.

# Листинг программы

In [1]:
import numpy as np
import math

## Функция алгоритма

In [2]:
def seidel_method(B, g, epsilon):
    x_k = np.copy(g) # Начальное приближение x_0 = g
    H, F = np.zeros((B.shape[0], B.shape[0])), np.zeros((B.shape[0], B.shape[0])) # Матрицы H и F инициализируем нулями
    x_k1 = np.zeros(x_k.shape[0]) # Столбец, отвечающий за k+1-ый шаг
    
    # Разбиение матрицы B = H + F
    for i in range(B.shape[0]):
        for j in range(B.shape[0]):
            if j < i: # Если находимся ниже главной диагонали
                H[i, j] = B[i, j] # Заполнение матрицы H значениями
            elif j >= i: # Если находимся на главной диагонали и выше
                F[i, j] = B[i, j] # Заполнение матрицы B значениями
                
    # Метод Зейделя в матричной форме
    iterations = 0 # Переменная для хранения числа итераций
    while True: # Пускаем цикл до тех пор, пока не выполнится условие остановки цикла
        iterations += 1 # Повышаем число итераций
        x_k1 = np.dot(H, x_k1) + np.dot(F, x_k) + g # Формула (3)
        if np.linalg.norm(x_k1 - x_k) <= epsilon: # Проверка условия остановки итерационного процесса
            break # Прерываем цикл
        x_k = np.copy(x_k1) # Переносим значение из (k+1)-ой итерации в k-ую
        
    return x_k1, iterations # Возвращаем значения k+1-ого шага и числа итераций

## Проверка результатов

1. При заданном $n$ сгенерировать случайным образом квадратную матрицу размера $n\times n$, имеющую диагональное доминирование. Элементы матрицы - действительные числа с двумя знаками после запятой.

In [3]:
from random import randint
 
n = int(input())
 
A = np.array([[randint(10, 1000) / 100 for _ in range(n)] for _ in range(n)]).astype(float)

summary = 0
for i in range(n):
    for j in range(n):
        if i != j:
            summary += math.fabs(A[i][j])
for i in range (n):
    if A[i][i] <= summary:
        A[i][i] = randint(int(summary)*100, 10000) / 100

print(*A, sep='\n')

 4


[96.56  5.84  1.21  6.91]
[ 8.9  91.73  0.96  1.24]
[ 0.49  7.28 72.22  2.07]
[ 5.26  7.57  0.4  63.95]


2. Сгенерировать случайным образом вектор решений $x$ (два знака после запятой).

In [4]:
x = np.array([randint(10, 1000) / 100 for _ in range(n)]).astype(float)

print(*x, sep='\n')

1.84
5.5
5.17
3.85


3. По сгенерированным данным вычислить вектор $b$ свободных членов системы по формуле $b = Ax$.

In [5]:
b = A.dot(x)

print(*b, sep='\n')

242.6496
530.6282
422.28849999999994
299.5889


4. В качестве условия для остановки итераций будем проверять $$| |x^{k+1} - x^k | |\leq \varepsilon,$$
где зададим $\varepsilon = 10^{-5}$.

In [6]:
epsilon = 1e-5

Введем в рассмотрение лемму и теорему:

**Лемма.** Модуль каждого собственного значения матриц не превосходит любой из ее норм, то есть $|\lambda_i(A)|\leq | | A | |.$

**Теорема**. Для сходимости метода простой итерации при любом начальном приближении $x^0$ необходимо и достаточно, чтобы все собственные значения матрицы $B$ были по модулю меньше единицы, то есть $|\lambda_i(B)| < 1$.

5. Для использования итерационного метода приведем исходную матрицу к виду $$x = Bx + g.$$ Для этого зададим $$B = E - \tau A,\quad g = \tau b.$$
Причем выберем $$\tau = \dfrac{1}{| |A| |}.$$ Потому как $$|\lambda_i(B)| = 1 - \dfrac{\lambda_i(A)}{| |A| |},\quad |\lambda_i(A)|\leq | |A| |,$$
то есть метод будем сходящимся по критерию сходимости метода простой итерации.

In [7]:
tau = 1 / np.linalg.norm(A)
B = np.eye(n) - A * tau

print(*B, sep='\n')

[ 0.41613954 -0.03531219 -0.0073164  -0.04178206]
[-0.05381481  0.44534466 -0.00580474 -0.00749779]
[-0.00296284 -0.04401931  0.56331398 -0.01251648]
[-0.03180516 -0.04577282 -0.00241864  0.61331943]


In [8]:
g = b * tau
print(*g, sep='\n')

1.4672069810923134
3.2085006503387943
2.5534129676496535
1.8114966006033681


6. Для полученных данных:

    a) найти решение системы указанным методом.

In [9]:
x_, iters = seidel_method(B, g, epsilon)

print(*x_, sep='\n')

1.8400023813288533
5.499999726278026
5.170000698352625
3.849988701889629


Число итераций равно:

In [10]:
print(iters)

24


Выполнить проверку путем вычисления вектора невязки.

In [11]:
r = np.zeros(n)
for i in range(n):
    for j in range(n):
        r[i] += A[i][j]*x_[j];
    r[i] -= b[i]

print(*r, sep='\n')

0.00015111764176367615
-1.725392826301686e-05
2.6222093367778143e-05
-0.0007117811027796961


б) найти матрицу, обратную к матрице системы.

Для этого в метод Зейделя в качестве параметров передадим матрицу $B$ вычисленную ранее и матрицу $$G = \tau E.$$

In [12]:
G = np.eye(n) * tau
print(*G, sep='\n')

[0.00604661 0.         0.         0.        ]
[0.         0.00604661 0.         0.        ]
[0.         0.         0.00604661 0.        ]
[0.         0.         0.         0.00604661]


Тогда матрица $A^{-1}$ имеет вид

In [13]:
A_, iters = seidel_method(B, G, epsilon)

print(*A_, sep='\n')

[ 0.01046926 -0.00056234 -0.00016168 -0.00111275]
[-0.00100615  0.01098401 -0.00012851 -0.00010036]
[ 5.11049862e-05 -1.06702060e-03  1.38596725e-02 -4.31514919e-04]
[-7.41020925e-04 -1.24398489e-03 -5.80674949e-05  1.57317314e-02]


Проверим, выполняется ли $$A^{-1} A = E.$$

In [16]:
E = np.dot(A_, A)


        
print(*E, sep='\n')

[ 9.99974718e-01 -4.32862789e-05  6.14605950e-06  1.50042693e-04]
[ 1.26694517e-05  9.99991697e-01  5.83498494e-06 -1.66597395e-05]
[-4.03148249e-05  5.25013073e-05  9.99810440e-01  1.24172953e-04]
[9.60081178e-05 2.28178965e-04 8.19726435e-06 9.99261028e-01]


в) найти число обусловленности.

In [17]:
nu = np.linalg.norm(A, 2) * np.linalg.norm(A_, 2)
print(nu)

1.6568156362565543
